<a href="https://colab.research.google.com/github/mayait/ClaseAnalisisDatos/blob/main/nlp/NLP_Tokenization_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Nombre del estudiante
Estudiante = "" #@param {type:"string"}
Código = "" #@param {type:"string"}

# NLP Procesamiento Natural de Lenguaje






*   Empecemos desde donde quedamos en el ultimo ejercicio, importemos el archivo de Excel que exportaste con un listado de Tweets.
* Asegurate que la columna con los tweets se llama **Tweet_Text**



In [ ]:
#!pip install wordcloud
import re
import unicodedata
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

import pandas as pd
from collections import Counter
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

import matplotlib.pyplot as plt
from wordcloud import WordCloud

import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
df = pd.read_excel('Spiderman.xlsx')
df.head()

-----

**Pasemos todo a minusculas**



In [ ]:
df['Tweet_Text'] = df['Tweet_Text'].str.lower()

In [ ]:
df.head()

----


# **Tokenizar**
En NLP el proceso de convertir las palabras o párrafos en inputs para la computadora se llama tokenización. Se puede pensar al token como la unidad para procesamiento semántico, es decir palabras.

## Tokenización con NLTK
El kit de herramientas de lenguaje natural, o más comúnmente NLTK, es un conjunto de bibliotecas y programas para el procesamiento del lenguaje natural (PLN) simbólico y estadísticos para el lenguaje de programación Python. NLTK incluye demostraciones gráficas y datos de muestra.

*Bird, Steven, Edward Loper and Ewan Klein (2009), Natural Language Processing with Python. O’Reilly Media Inc.*
[nltk.org](https://nltk.org/)

**NLTK tiene librerias que permiten tokenizar, veamos la diferencia entre ellas**

TreeBankTokenizer es perfecto en partir el texto, sin embargo no es bueno con Tweets, quita información como hashtags y menciones.


In [ ]:
import nltk
from nltk.tokenize import WordPunctTokenizer 
from nltk.tokenize import TreebankWordTokenizer 

texto = "¿Cuánto tiempo pasó desde que comí una manzana?"
tweet = ' #Oscar a Mejor Película Internacional; 😩😔😁 un filme ecuatoriano figura en la lista » https://t.co/cb5p6eX3DZ https://t.co/kQ0nVAbyOn'

print(WordPunctTokenizer().tokenize(texto))
print(WordPunctTokenizer().tokenize(tweet))
print(TreebankWordTokenizer().tokenize(tweet))
print(WordPunctTokenizer().tokenize(tweet))


## NLTK Tweet Tokenizer

Por suerte NLTK ya tiene una libreria que soluciona este problema y tokeniza manteniendo emoticonos, menciones y hashtags

In [ ]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

tweet = ' #Oscar a Mejor Película Internacional; 😩😔😁 un filme ecuatoriano figura en la lista » https://t.co/cb5p6eX3DZ https://t.co/kQ0nVAbyOn'
tweet_tokenizer.tokenize(tweet)

# Creamos una lista de palabras y sus frecuencias

In [ ]:
import nltk
from collections import Counter
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

# Una lista para todas las palabras
palabras_todas = []

# Creamos un contador de frecuencias
count_all = Counter()

for tweet in df["Tweet_Text"]:
  # Crear una lista de todos los terminos con NLTK
  terms_all = tweet_tokenizer.tokenize(tweet)
  # Actualizar el contador
  count_all.update(terms_all)
  # Aumentar al listado de todas las palabras
  palabras_todas.extend(terms_all)

# Las primeras 10 palabras más comunes
print("Cantidad de terminos unicos: ", len(count_all))
print("Total de terminos (incluye repetidos): ", len(palabras_todas))
count_all.most_common(20)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

unique_string=(" ").join(palabras_todas)
wordcloud = WordCloud(width = 600, height = 300, background_color="white",).generate(unique_string)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("wordcloud_polluted"+".png", bbox_inches='tight')
plt.show()
plt.close()

## Quitar StopWords o Palabras Vacias

Si vemos en el ejercicio anterior, hay muchas palabras con frecuencias altas que no nos dicen nada, estas son palabras vacias o Stopwords.

Las palabras vacías son aquellas palabras en lenguaje natural que tienen muy poco significado, como «es», «una», «el», etc. 

Las palabras vacías a menudo se eliminan del texto antes de entrenar los modelos de aprendizaje profundo y de Machine Learning, ya que las palabras vacías ocurren en abundancia, por lo que brindan poca o ninguna información única que se pueda usar para la clasificación o agrupación.

[Palabras Vacias Wikipedia](https://es.wikipedia.org/wiki/Palabra_vac%C3%ADa)

NLTK tiene un listado de palabras vacias en ingles y español, pero tambien podemos añadir las nuestras.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

otros_a_quitar = ['»', ';', ',', '.','!','?','¿',',','rt', 'via' ]
 
punctuation = list(string.punctuation)
stop = stopwords.words('spanish') + stopwords.words('english') + punctuation
print(stop)
print('Total de stop words: ', len(stop))

In [ ]:
# Listas de frecuencias
count_stop = Counter()
count_hash = Counter()
count_mentions = Counter()
count_terms_only = Counter()

# Listas de palabras
lista_todas = []
lista_hash = []
lista_mentions = []
lista_terms_only = []

for tweet in df["Tweet_Text"]:

  # Crear una lista de todos los terminos que no están en el stop
  terms_stop = [term for term in tweet_tokenizer.tokenize(tweet) if term not in stop]
  # Actualizar el contador
  count_stop.update(terms_stop)
  # Actualizar la lista
  lista_todas.extend(terms_stop)

  # Contar hashtags
  terms_hash = [term for term in tweet_tokenizer.tokenize(tweet) if term.startswith('#')]
  # Actualizar el contador
  count_hash.update(terms_hash)
  # Actualizar la lista
  lista_hash.extend(terms_hash)

  # Contar menciones
  terms_mentions = [term for term in tweet_tokenizer.tokenize(tweet) if term.startswith('@')]
  # Actualizar el contador
  count_mentions.update(terms_mentions)
  # Actualizar la lista
  lista_mentions.extend(terms_mentions)

  # Contar terms sin hash ni menciones
  terms_only = [term for term in tweet_tokenizer.tokenize(tweet) if term not in stop and not term.startswith(('#', '@', 'http'))]
  # Actualizar el contador
  count_terms_only.update(terms_only)
  # Actualizar la lista
  lista_terms_only.extend(terms_only)

print(count_stop.most_common(20))
print(count_hash.most_common(20))
print(count_terms_only.most_common(20))
print(count_mentions.most_common(20))

# WordCloud

Una nube de palabras o nube de etiquetas es una representación visual de las palabras que conforman un texto, en donde el tamaño es mayor para las palabras que aparecen con más frecuencia.

Uno de sus usos principales es la visualización de las etiquetas de un sitio web, de modo que los temas más frecuentes en el sitio se muestren con mayor prominencia. 

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

unique_string=(" ").join(lista_terms_only)
wordcloud = WordCloud(width = 600, height = 300, background_color="white",).generate(unique_string)
plt.figure(figsize=(30,16))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("wordcloud"+".png", bbox_inches='tight')
plt.show()
plt.close()

# WordCloud con Mascara

Python nos permite componer un wordcloud usando una imagen como mascara, asegurate de buscar una foto con fondo blanco.
**No olvides subir la imagen spiderman_mask.jpg junto al cuaderno.**

In [ ]:
# Start with loading all necessary libraries
import numpy as np
import pandas as pd
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import requests
import matplotlib.pyplot as plt

unique_string=(" ").join(lista_terms_only)

# Utilizar una imagen del internet como mascara
#url_imagen_original = "https://i.pinimg.com/originals/80/7b/29/807b29f756cc47621f92b3581e263272.jpg"
#mask = np.array(Image.open( requests.get(urlimagen_original, stream=True).raw ))

# Abrir una imagen de mascara local
mask = np.array(Image.open("spiderman_mask.jpg") )

wordcloud_por = WordCloud(background_color="white", max_words=5000, mask=mask).generate(unique_string)

# create coloring from image
image_colors = ImageColorGenerator(mask)
plt.figure(figsize=[20,20])
plt.imshow(wordcloud_por.recolor(color_func=image_colors), interpolation="bilinear")
plt.axis("off")


# store to file
plt.savefig("spiderman_wordcloud.png", format="png")
plt.show()

# Exportar listas a Excel para análisis posterior

**Creamos una función para exportar la lista de frecuencias de palabras a Excel**

In [ ]:
def counter_to_excel(counter, filename):
  palabras = []
  for i in counter.items():
    palabras.append([i[0],i[1]])
  pd.DataFrame(palabras).to_excel(filename, header=False, index=False)
  return None

Extraer la data a excel para procesarla


In [ ]:
counter_to_excel(count_stop, "count_stop.xlsx")
counter_to_excel(count_hash, "count_hash.xlsx")
counter_to_excel(count_terms_only, "count_terms_only.xlsx")
counter_to_excel(count_mentions, "count_mentions.xlsx")

pd.DataFrame(lista_todas).to_excel('lista_todas.xlsx', header=False, index=False)
pd.DataFrame(lista_hash).to_excel('lista_hash.xlsx', header=False, index=False)
pd.DataFrame(lista_mentions).to_excel('lista_mentions.xlsx', header=False, index=False)
pd.DataFrame(lista_terms_only).to_excel('lista_terms_only.xlsx', header=False, index=False)

# Análisis de senimiento
![alt text](https://ai2-s2-public.s3.amazonaws.com/figures/2017-08-08/0cc39c70021d21fc0d2fd7a986ccd242bef86c29/6-Figure1-1.png)

In [ ]:
import seaborn as sns

sns.set(rc={'figure.figsize':(30,1)})

def visualise_sentiments(data):
  sns.heatmap(pd.DataFrame(data).set_index("Sentence").T,center=0, annot=True, cmap = "PiYG")

def visualise_sentiments_from_sentence(sentence):
  visualise_sentiments({
      "Sentence":["SENTENCE"] + sentence.split(),
      "Sentiment":[sid.polarity_scores(sentence)["compound"]] + [sid.polarity_scores(word)["compound"] for word in sentence.split()]
  })

## NLTK Vader

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
sentence = "i really like you but you can be so dumb some times"

In [ ]:
sid.polarity_scores(sentence)

In [ ]:
visualise_sentiments_from_sentence(sentence)

In [ ]:
# Que pasa si le pasamos texto en español
visualise_sentiments_from_sentence(tweet)

# Traducir
Casi todos los módelos de Analisis de sentimiento funcionan en Ingles, una solución es traducir.

In [ ]:
!pip install -q spanish-sentiment-analysis
!pip install -q googletrans==3.1.0a0

In [ ]:
from googletrans import Translator
from time import sleep

def traducir(text, dest='en', **kwargs):
    translator = Translator()
    result = None
    while result == None:
        try:
            result = translator.translate(text,**kwargs)
        except Exception as e:
            print(e)
            translator = Translator()
            sleep(0.5)
            pass
    return result.text.lower()

In [ ]:
english_tweet = traducir(tweet)
print('ES', tweet)
print('ES', english_tweet)

In [ ]:
sid.polarity_scores(tweet)

In [ ]:
visualise_sentiments_from_sentence(english_tweet)

***Hagamos un ejemplo más claro***

In [ ]:
tweet_negativo = "@cnt son la peor operadora del mundo, ojalá se quiebren pronto y dejen de estafar a las personas"

In [ ]:
sid.polarity_scores(tweet_negativo)

In [ ]:
visualise_sentiments_from_sentence(tweet_negativo)

In [ ]:
negative_tweet = traducir(tweet_negativo)
negative_tweet

In [ ]:
sid.polarity_scores(negative_tweet)

In [ ]:
visualise_sentiments_from_sentence(negative_tweet)

In [ ]:
small = df.sample(5)
small

In [ ]:
df['tweet_traducido'] = df.Tweet_Text.apply(traducir)

In [ ]:
df

In [ ]:
df.tweet_traducido.apply(sid.polarity_scores)

Creamos una función para obtener solo el valor compound de la polaridad y añadirlo a una columna del dataframe.

In [ ]:
def get_polarity_score_compound(text):
    return sid.polarity_scores(text)['compound']

In [ ]:
df['sentimiento'] = df.tweet_traducido.apply( get_polarity_score_compound )

In [ ]:
df.sentimiento.hist()

# Ejercicio

## 1. 
Importa un excel con un listado de Tweets en inglés y almacenalos en un dataframe llamado df_tuits.
Utiliza el cuaderno de [scrapping en twitter](https://colab.research.google.com/github/mayait/ClaseAnalisisDatos/blob/main/nlp/2022_Twitter_Data_Scrapper.ipynb)

In [ ]:
# df_tuits = pd.read_excel...

## 2.
¿Cúales son las palabras más comunes de esa busqueda?
Elabora una nube de palabras

## 3.
- Utilizando tuits descargados en ingles
- ¿Cuantos tuits son posittivos?
- ¿Cuantos son negativos?
- Cúal es el sentimiento de la conversación

## 4. Elabora un histograma del sentimiento de los tuits